# Preprocess the data

## Rigid registration and skull extraction

In [1]:
import os
from os.path import expanduser as eu
from headctools.registration import register_file_folder
from headctools.preprocessing import ct_skull, utils

In [3]:
imgs = eu('~/Code/datasets/cq500/converted/selected/test')
fixed_img = os.path.join(imgs, 'CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz')
# Registered images will be saved in a subfolder with the name of the fixed img

params = {'reg_atlas_path': fixed_img,
          'overwrite_images': False,
          # 'skip_registration': True  # Non-default params
          }

ct_skull(imgs, params = params)  # Preprocess the imgs

Getting nii.gz images from /home/franco/Code/datasets/cq500/converted/selected/test
Found 5 images
 No save path specified. Using default save path.
Preprocessing {'image': '/home/franco/Code/datasets/cq500/converted/selected/test/CQ500-CT-216_CT 0.625mm.nii.gz'}
  File /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-216_CT 0.625mm.nii.gz already exists..
Preprocessing {'image': '/home/franco/Code/datasets/cq500/converted/selected/test/CQ500-CT-390_CT 5mm POST CONTRAST.nii.gz'}
  File /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-390_CT 5mm POST CONTRAST.nii.gz already exists..
Preprocessing {'image': '/home/franco/Code/datasets/cq500/converted/selected/test/CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz'}
  File /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz already exists..
Preprocessing {'image': '/home/franco/Code/datasets/cq5

'/home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull'

## Mesh atlas construction

### Decimate meshes

In [2]:
meshes_folder = '~/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull'
decimate_factor = 0.01
saved_meshes = utils.decimate_meshes(meshes_folder, decimate_factor,
                                     overwrite=False)

Decimating meshes...
  Input folder: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull
  Found 5 meshes
    [1/5] Input mesh: CQ500-CT-390_CT 5mm POST CONTRAST.stl
    Mesh already exists, skipping...
    [2/5] Input mesh: CQ500-CT-312_CT PRE CONTRAST THIN.stl
    Mesh already exists, skipping...
    [3/5] Input mesh: CQ500-CT-405_CT Thin Plain.stl
    Mesh already exists, skipping...
    [4/5] Input mesh: CQ500-CT-216_CT 0.625mm.stl
    Mesh already exists, skipping...
    [5/5] Input mesh: CQ500-CT-184_CT Thin Plain.stl
    Mesh already exists, skipping...


## Deformetrica
### Atlas estimation

In [3]:
from src.deformetrica import estimate_atlas, estimate_registration
template_path, dataset_paths = saved_meshes[0], saved_meshes[1:]

In [4]:
estimate_atlas(template_path, dataset_paths)

Logger has been set to: INFO
>> No initial CP spacing given: using diffeo kernel width of 40.0
OMP_NUM_THREADS was not found in environment variables. An automatic value will be set.
OMP_NUM_THREADS will be set to 2
>> No specified state-file. By default, Deformetrica state will by saved in file: output/deformetrica-state.p.
>> Set of 80 control points defined.
>> Momenta initialized to zero, for 4 subjects.
>> Started estimator: GradientAscent


/home/franco/anaconda3/envs/msr/lib/python3.8/site-packages/deformetrica/core/observations/deformable_objects/landmarks/surface_mesh.py:37: UserWarning:

This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)



------------------------------------- Iteration: 0 -------------------------------------
>> Log-likelihood = -4.609E+05 	 [ attachment = -4.609E+05 ; regularity = 0.000E+00 ]
>> Step size and gradient norm: 
		3.349E-06   and   2.986E+05 	[ landmark_points ]
		6.220E-05   and   1.608E+04 	[ momenta ]
------------------------------------- Iteration: 1 -------------------------------------
>> Log-likelihood = -4.439E+05 	 [ attachment = -4.439E+05 ; regularity = -3.107E+00 ]
>> Step size and gradient norm: 
		5.024E-06   and   2.927E+05 	[ landmark_points ]
		9.330E-05   and   1.577E+04 	[ momenta ]
------------------------------------- Iteration: 2 -------------------------------------
>> Log-likelihood = -4.195E+05 	 [ attachment = -4.195E+05 ; regularity = -1.895E+01 ]
>> Step size and gradient norm: 
		7.536E-06   and   2.833E+05 	[ landmark_points ]
		1.400E-04   and   1.527E+04 	[ momenta ]
------------------------------------- Iteration: 3 -------------------------------------
>> 

In [4]:
estimate_registration(template_path, dataset_paths)

Logger has been set to: INFO
>> No initial CP spacing given: using diffeo kernel width of 40.0
OMP_NUM_THREADS was not found in environment variables. An automatic value will be set.
OMP_NUM_THREADS will be set to 2
>> No specified state-file. By default, Deformetrica state will by saved in file: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/reg_CQ500-CT-390_CT 5mm POST CONTRAST_decimated_1perc/deformetrica-state.p.
>> Set of 80 control points defined.
>> Momenta initialized to zero, for 4 subjects.
>> Started estimator: GradientAscent


/home/franco/anaconda3/envs/msr/lib/python3.8/site-packages/deformetrica/core/observations/deformable_objects/landmarks/surface_mesh.py:37: UserWarning:

This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)



------------------------------------- Iteration: 0 -------------------------------------
>> Log-likelihood = -4.609E+05 	 [ attachment = -4.609E+05 ; regularity = 0.000E+00 ]
>> Step size and gradient norm: 
		6.220E-05   and   1.608E+04 	[ momenta ]
------------------------------------- Iteration: 1 -------------------------------------
>> Log-likelihood = -4.450E+05 	 [ attachment = -4.450E+05 ; regularity = -3.107E+00 ]
>> Step size and gradient norm: 
		9.330E-05   and   1.579E+04 	[ momenta ]
------------------------------------- Iteration: 2 -------------------------------------
>> Log-likelihood = -4.221E+05 	 [ attachment = -4.221E+05 ; regularity = -1.899E+01 ]
>> Step size and gradient norm: 
		1.400E-04   and   1.532E+04 	[ momenta ]
------------------------------------- Iteration: 3 -------------------------------------
>> Log-likelihood = -3.900E+05 	 [ attachment = -3.899E+05 ; regularity = -6.610E+01 ]
>> Step size and gradient norm: 
		2.099E-04   and   1.457E+04 	[ mom